In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import satpy
from satpy.readers.utils import get_geostationary_angle_extent

In [3]:
from pathlib import Path

In [19]:
import numpy as np
import pandas as pd
import xarray as xr

import netCDF4

from tqdm import tqdm

import cartopy.crs as ccrs

import warnings

from utils import spherical_angle_add, ALL_CHANNELS, AHI_BANDS, ABI_BANDS, remap

In [9]:
%load_ext autoreload
%autoreload 2

In [11]:
%aimport make_sample

In [23]:
dt = '20200930T0200'

In [26]:
make_sample.main(dt)

Compositing: 100%|██████████| 3/3 [00:13<00:00,  4.37s/it]        


Saving composite_cache/20200930T0200/temp_11_00um.nc
Took 16.7 sec


Compositing: 100%|██████████| 3/3 [00:30<00:00, 10.27s/it]       


Saving composite_cache/20200930T0200/refl_00_47um.nc
Took 34.3 sec


  0%|          | 0/3 [00:00<?, ?it/s]

...

Took 16.1 sec


Compositing: 100%|██████████| 3/3 [00:15<00:00,  5.31s/it]        


Saving composite_cache/20200930T0200/temp_12_00um.nc
Took 19.4 sec


Compositing: 100%|██████████| 3/3 [00:15<00:00,  5.21s/it]        


Saving composite_cache/20200930T0200/temp_13_30um.nc
Took 19.1 sec


In [9]:
L1B = Path('l1b')
INDEX = Path('index')

In [11]:
COMP_CACHE = Path('composite_cache')

In [12]:
grid_width = int(360/.05)
grid_height = int(180/.05)
grid_shape = grid_height, grid_width

In [20]:
wmo_ids = xr.open_dataset(COMP_CACHE / 'wmo_id.nc').wmo_id

In [24]:
encoding = {
    'zlib':True,
    'dtype':'i2',
    '_FillValue':netCDF4.default_fillvals['i2'],
    'scale_factor':.01,
    'add_offset':50
}

In [31]:
for k in ALL_CHANNELS:
    f = COMP_CACHE / dt/ f'{k}.nc'
    if f.is_file():
        f.unlink()
        print(f)

composite_cache/20200925T1800/temp_11_00um.nc


In [32]:
ordered_channels = ['temp_11_00um', *sorted(ALL_CHANNELS - set('temp_11_0um'))]
for channel in ordered_channels:
    out_dir = COMP_CACHE / dt
    out_dir.mkdir(exist_ok=True)
    out_nc = out_dir / f'{channel}.nc'
#     if out_nc.is_file():
#         print(f'Already have {out_nc}')
#         continue
    start = time.time()
    composite = xr.DataArray(np.full((3, *grid_shape), np.nan, dtype=np.float32), dims=['layer','lat','lon'])
    with tqdm([('g16','abi_l1b', ABI_BANDS, 152),
               ('g17','abi_l1b', ABI_BANDS, 664),
               ('h8','ahi_hsd', AHI_BANDS, 167)]) as bar:
        for sat, reader, band_map, wmo_id in bar:
            if channel in band_map:
                band = band_map[channel]
            else:
                print(f'{sat} has no {channel}')
                continue
            if sat.startswith('g'):
                index_band = {1:1,3:1,5:1,2:2}.get(band, 14)
            else:
                index_band = {1:1,2:1,3:3,4:1}.get(band, 14)
            dt_dir = L1B/sat/dt
            index_dir = INDEX / sat / f'{index_band:02d}'
            src_index = np.memmap(index_dir / 'src_index.dat', mode='r', dtype=np.uint64)
            dst_index = np.memmap(index_dir / 'dst_index.dat', mode='r', dtype=np.uint64)
            assert index_dir.is_dir()
            band_dir = dt_dir / f'{band:02d}'
            assert band_dir.is_dir()
            files = list(band_dir.glob('*'))
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                scene = satpy.Scene(files, reader=reader)
                ds_names = scene.available_dataset_names()
                scene.load(ds_names)
                area = scene[ds_names[0]].area
                bar.set_description(f'Loading {sat} band {band}')
                v = scene[ds_names[0]].values
            bar.set_description('Remapping imagery')
            out = remap(src_index, dst_index, v, grid_shape)
            scene.unload()
            bar.set_description('Compositing')
            for layer in range(composite.shape[0]):
                mask = wmo_ids[layer].values == wmo_id
                composite.values[layer, mask] = out[mask]
    print(f"Saving {out_nc}")
    composite.to_dataset(name=channel).to_netcdf(out_nc, encoding={channel:encoding})
    end = time.time()
    dur = end - start
    print(f'Took {dur:.1f} sec')

Compositing: 100%|██████████| 3/3 [00:12<00:00,  4.11s/it]        


Saving composite_cache/20200925T1800/temp_11_00um.nc
Took 15.9 sec


Compositing: 100%|██████████| 3/3 [00:25<00:00,  8.66s/it]       


Saving composite_cache/20200925T1800/refl_00_47um.nc
Took 29.6 sec


  0%|          | 0/3 [00:00<?, ?it/s]

...

Saving composite_cache/20200925T1800/temp_11_00um.nc
Took 16.0 sec


Compositing: 100%|██████████| 3/3 [00:13<00:00,  4.34s/it]        


Saving composite_cache/20200925T1800/temp_12_00um.nc
Took 17.0 sec


Compositing: 100%|██████████| 3/3 [00:13<00:00,  4.60s/it]        


Saving composite_cache/20200925T1800/temp_13_30um.nc
Took 17.6 sec
